Importing Libraries

In [106]:
import pandas as pd

Loading data

In [107]:
job_postings = pd.read_csv('data/job_postings.csv')

companies = pd.read_csv('data/company_details/companies.csv')
company_industries = pd.read_csv('data/company_details/company_industries.csv')
company_specialities = pd.read_csv('data/company_details/company_specialities.csv')
company_employee_count = pd.read_csv('data/company_details/employee_counts.csv')

mapping_skills = pd.read_csv('data/maps/skills.csv')

job_skills = pd.read_csv('data/job_details/job_skills.csv')
job_salaries = pd.read_csv('data/job_details/salaries.csv')

Dataset Summary

In [108]:
unique_values = {
    'job_postings': job_postings['job_id'].nunique(), # details of job postings
    'job_skills': job_skills['job_id'].nunique(), # job posting with the required skill abbreviated
    'companies': companies['company_id'].nunique(), # information on the company
    'skills': mapping_skills['skill_name'].nunique() # maps between skill abbreviations and full skill names
    }
unique_values

{'job_postings': 33246, 'job_skills': 32422, 'companies': 11361, 'skills': 35}

### Preprocessing

##### Preprocessing Job Postings

In [109]:
job_postings.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


Dropping not-necessary columns

In [110]:
columns_to_drop = ['work_type','application_url','applies','original_listed_time','application_type','max_salary','med_salary','min_salary','pay_period','posting_domain','sponsored','expiry','remote_allowed','views','closed_time', 'skills_desc', 'listed_time', 'currency', 'compensation_type','scraped']
job_postings= job_postings.drop(columns=columns_to_drop,axis=1)

Converting company_id from float to int

In [111]:
if 'company_id' in job_postings.columns:
    job_postings['company_id'] = job_postings['company_id'].astype('Int64')

Renaming and reordering columns

In [112]:
job_postings = job_postings.rename(columns={'formatted_work_type':'work_type','formatted_experience_level'
                                            :'experience_level'})
job_postings = job_postings.sort_values('job_id', ascending=False)

order = ['job_id','company_id','title', 'description', 'work_type','experience_level', 'location', 'job_posting_url']
job_postings = job_postings[order]

Checking for null values

In [113]:
missing_data = job_postings.isnull().sum()
significant_missing_columns = missing_data[missing_data > 0].sort_values(ascending=True)
significant_missing_columns

description            1
company_id           654
experience_level    9181
dtype: int64

Dropping null values

In [114]:
job_postings.dropna(subset=['company_id','description'], inplace=True)
job_postings.shape

(32591, 8)

Missing Experience level is filled with "Not Specified"

In [115]:
col_experience_level= ['experience_level']
for col in col_experience_level:
    job_postings[col].fillna("Not Specified", inplace=True)

Removing job postings where company is not existent in companies table

In [116]:
job_postings = job_postings[job_postings['company_id'].isin(companies['company_id'])]

Saving preprocessed job_postings

In [117]:
job_postings.to_csv('data_preprocessed/job_postings.csv', index=False)

##### Preprocessing Companies

In [118]:
companies.head()

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare
2,1021,GE Power,"GE Power, part of GE Vernova, is a world energ...",7.0,NY,US,Schenectady,12345,1 River Road,https://www.linkedin.com/company/gepower
3,1025,Hewlett Packard Enterprise,Official LinkedIn of Hewlett Packard Enterpris...,7.0,Texas,US,Houston,77389,1701 E Mossy Oaks Rd Spring,https://www.linkedin.com/company/hewlett-packa...
4,1028,Oracle,We’re a cloud technology company that provides...,7.0,Texas,US,Austin,78741,2300 Oracle Way,https://www.linkedin.com/company/oracle


Dropping not-neccesary columns

In [119]:
columns_to_drop = ['zip_code','description','company_size']
companies= companies.drop(columns=columns_to_drop,axis=1)

Checking for '0'

In [120]:
zero_counts_by_column = (companies.applymap(lambda x: str(x) == '0')).sum()
print("\nNumber of occurrences of '0' in each column:")
print(zero_counts_by_column)


Number of occurrences of '0' in each column:
company_id       0
name             0
state          928
country        311
city           438
address       1658
url              0
dtype: int64


C:\Users\Blina Smailaj\AppData\Local\Temp\ipykernel_30756\1247404864.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  zero_counts_by_column = (companies.applymap(lambda x: str(x) == '0')).sum()


Replacing '0' with 0

In [121]:
companies[['state', 'country', 'city', 'address']] = companies[['state', 'country', 'city', 'address']].replace('0', pd.NA)

Null values in companies

In [122]:
missing_data = companies.isnull().sum()
significant_missing_columns = missing_data[missing_data > 0].sort_values(ascending=True)
significant_missing_columns

name          1
country     311
city        439
state       936
address    1669
dtype: int64

Filling null data with '-'

In [123]:
companies = companies.fillna('-')

Saving companies preprocessed

In [124]:
companies.to_csv('data_preprocessed/companies.csv', index=False)

##### Preprocessing Job Skills

In [125]:
job_skills.head()

,job_id,skill_abr
0,3690843087,ACCT
1,3690843087,FIN
2,3691763971,MGMT
3,3691763971,MNFC
4,3691775263,MGMT


In [126]:
mapping_skills.head()

,skill_abr,skill_name
0,PRCH,Purchasing
1,SUPL,Supply Chain
2,PR,Public Relations
3,SCI,Science
4,STRA,Strategy/Planning


In [127]:
job_skills = pd.merge(job_skills, mapping_skills, on='skill_abr', how='left')
job_skills = job_skills.drop('skill_abr', axis=1)
job_skills.head()

,job_id,skill_name
0,3690843087,Accounting/Auditing
1,3690843087,Finance
2,3691763971,Management
3,3691763971,Manufacturing
4,3691775263,Management


In [128]:
job_skills.shape

(56591, 2)

In [129]:
job_skills.head()

,job_id,skill_name
0,3690843087,Accounting/Auditing
1,3690843087,Finance
2,3691763971,Management
3,3691763971,Manufacturing
4,3691775263,Management


Ensuring that no skills are associated with a posting that doesn't exist

In [130]:
missing_job_postings_ids = job_skills.loc[~job_skills['job_id'].isin(job_postings['job_id']), 'job_id']

num_missing_companies = len(missing_job_postings_ids)
print("Number of missing job postings:", num_missing_companies)

Number of missing job postings: 1833


In [131]:
job_skills = job_skills[~job_skills['job_id'].isin(missing_job_postings_ids)]

Saving job_skills preprocessed

In [132]:
job_skills.to_csv('data_preprocessed/job_skills.csv', index=False)

##### Preprocessing Company Industries

In [133]:
company_industries.head()

,company_id,industry
0,81149246,Higher Education
1,10033339,Information Technology & Services
2,6049228,Accounting
3,2641066,Electrical & Electronic Manufacturing
4,96649998,Marketing & Advertising


Ensuring that no industries are associated with a company that doesn't exist

In [134]:
missing_company_ids = company_industries.loc[~company_industries['company_id'].isin(companies['company_id']), 'company_id']

num_missing_companies = len(missing_company_ids)
print("Number of missing companies:", num_missing_companies)

Number of missing companies: 45


Drop rows with missing companies from company_industries

In [135]:
company_industries = company_industries[~company_industries['company_id'].isin(missing_company_ids)]

Saving company_industries preprocessed

In [136]:
company_industries.to_csv('data_preprocessed/company_industries.csv', index=False)

##### Preprocessing company specialities

In [137]:
company_specialities.head()

,company_id,speciality
0,81149246,Childrens Music Education
1,81149246,Foundational Music Theory
2,81149246,Child Music Lessons
3,81149246,social emotional learning
4,81149246,social emotional development


Ensuring that no specialities are associated with a company that doesn't exist

In [138]:
missing_company_ids = company_specialities.loc[~company_specialities['company_id'].isin(companies['company_id']), 'company_id']

num_missing_companies = len(missing_company_ids)
print("Number of missing companies:", num_missing_companies)

Number of missing companies: 335


Drop rows with missing companies from company_specialities

In [139]:
company_specialities = company_specialities[~company_specialities['company_id'].isin(missing_company_ids)]

Removing duplicate combinations

In [140]:
company_specialities['speciality'] = company_specialities['speciality'].str.capitalize().str.strip()
company_specialities.drop_duplicates(subset=['company_id', 'speciality'], keep='first', inplace=True)

Removing rows where speciality is too large

In [141]:
company_specialities = company_specialities[company_specialities['speciality'].apply(len) <= 200]

Saving company_specialities preprocessed

In [142]:
company_specialities.to_csv('data_preprocessed/company_specialities.csv', index=False)

##### Preprocessing employee_counts

In [143]:
company_employee_count.head()

,company_id,employee_count,follower_count,time_recorded
0,81149246,6,91,1.692645e+09
1,10033339,3,187,1.692645e+09
2,6049228,20,82,1.692645e+09
3,2641066,45,2336,1.692645e+09
4,96649998,0,2,1.692645e+09


Ensuring that no count of employes that are associated with a company that doesn't exist

In [144]:
missing_company_ids = company_employee_count.loc[~company_employee_count['company_id'].isin(companies['company_id']), 'company_id']

num_missing_companies = len(missing_company_ids)
print("Number of missing companies:", num_missing_companies)

Number of missing companies: 45


Removing missing company_id's

In [145]:
company_employee_count = company_employee_count[~company_employee_count['company_id'].isin(missing_company_ids)]

Keeping only last recorded company follower count

In [146]:
#company_employee_count.sort_values('time_recorded', ascending=False, inplace=True)
#company_employee_count = company_employee_count.drop_duplicates('company_id', keep='first')


In [147]:
company_employee_count.dtypes

company_id          int64
employee_count      int64
follower_count      int64
time_recorded     float64
dtype: object

Converting time_recorded to int

In [148]:
company_employee_count['time_recorded'] = company_employee_count['time_recorded'].astype(int)

Saving company_employee_counts preprocessed 

In [149]:
company_employee_count.to_csv('data_preprocessed/employee_counts.csv', index=False)

##### Preprocessing Job Salaries

In [150]:
job_salaries.head()

,salary_id,job_id,max_salary,med_salary,min_salary,pay_period,currency,compensation_type
0,1,3378133231,30.0,NaN,22.0,HOURLY,USD,BASE_SALARY
1,2,3690843087,65000.0,NaN,55000.0,YEARLY,USD,BASE_SALARY
2,3,3691794313,22.0,NaN,19.0,HOURLY,USD,BASE_SALARY
3,4,3691795389,70000.0,NaN,68000.0,YEARLY,USD,BASE_SALARY
4,5,3691797089,22.0,NaN,18.0,HOURLY,USD,BASE_SALARY


Checking for null values

In [151]:
missing_data = job_salaries.isnull().sum()
significant_missing_columns = missing_data[missing_data > 0].sort_values(ascending=True)
significant_missing_columns

max_salary     2241
min_salary     2241
med_salary    11111
dtype: int64

Preprocess Min, Med and Max Salaries

In [152]:
def process_salaries(df):
    # Calculate med_salary where it's NaN and both min_salary and max_salary exist
    df.loc[df['med_salary'].isna() & df['min_salary'].notna() & df['max_salary'].notna(), 'med_salary'] = (df['min_salary'] + df['max_salary']) / 2

    # Drop the min_salary and max_salary columns
    df = df.drop(['min_salary', 'max_salary'], axis=1)

    # Replace NaN values in med_salary with a float value, for example 0.0
    df['med_salary'] = df['med_salary'].fillna(0.0)

    return df

job_salaries = process_salaries(job_salaries)

In [153]:
job_salaries.dtypes

salary_id              int64
job_id                 int64
med_salary           float64
pay_period            object
currency              object
compensation_type     object
dtype: object

In [154]:
job_salaries.head()

,salary_id,job_id,med_salary,pay_period,currency,compensation_type
0,1,3378133231,26.0,HOURLY,USD,BASE_SALARY
1,2,3690843087,60000.0,YEARLY,USD,BASE_SALARY
2,3,3691794313,20.5,HOURLY,USD,BASE_SALARY
3,4,3691795389,69000.0,YEARLY,USD,BASE_SALARY
4,5,3691797089,20.0,HOURLY,USD,BASE_SALARY


Ensuring that no salaries are associated with a posting that doesn't exist

In [155]:
missing_job_postings_ids = job_salaries.loc[~job_salaries['job_id'].isin(job_postings['job_id']), 'job_id']

num_missing_job_postings_ids = len(missing_job_postings_ids)
print("Number of missing job postings:", num_missing_job_postings_ids)

Number of missing job postings: 236


In [156]:
job_salaries = job_salaries[~job_salaries['job_id'].isin(missing_job_postings_ids)]

Saving job_salaries preprocessed

In [157]:
job_salaries.to_csv('data_preprocessed/job_salaries.csv', index=False)